In [1]:
from pytrends.request import TrendReq
from addon import *
import pandas as pd
import time
import pycountry

In [30]:

pytrends = TrendReq(hl='en-US', tz=360)

## %
kw_list = ["do i have covid"]
df = pd.DataFrame()

In [31]:
states = [ 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'IL', 'IN', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [32]:
i = 0
while i < len(states):
    try:
        pytrends.build_payload(kw_list, cat=0, timeframe='2020-03-22 2020-10-09', geo="US-" + states[i], gprop='')
        
        state_df = pytrends.interest_over_time()
        state_df["State"] = states[i]
        
        df = pd.concat([df, state_df])
        time.sleep(.1)

        print(states[i])
        i += 1

        time.sleep(3)
    except Exception as e:
        i -= 1
        time.sleep(5)
        print(e)


AL
AZ
AR
CA
CO
CT
DE
DC
FL
GA
IL
IN
KS
KY
LA
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OK
OR
PA
RI
SC
SD
TN
TX
VT
VA
WA
WV
WI
WY


In [33]:
df.to_csv("../../data/trends/final_data/USA_statewise_2.csv")

In [36]:
df1 = pd.read_pickle("../../df1.pkl")
df2 = pd.read_pickle("../../df2.pkl")
df3 = pd.read_pickle("../../df3.pkl")
df4 = pd.read_pickle("../../df4.pkl")

df_combined = pd.concat([df1, df2, df3, df4])
df_combined = df_combined.sort_values(by='date')
df_combined["date"] = df_combined["date"] + " 00:00:01"
df_combined["date"] = pd.to_datetime(df_combined["date"])
print(df_combined)
df_combined.to_csv("../../data/hopkins_data/df_new.csv")

                   date     FIPS     Admin2  Province_State Country_Region  \
820 2020-03-22 00:00:01  45001.0  Abbeville  South Carolina             US   
0   2020-03-23 00:00:01  45001.0  Abbeville  South Carolina             US   
0   2020-03-24 00:00:01  45001.0  Abbeville  South Carolina             US   
0   2020-03-25 00:00:01  45001.0  Abbeville  South Carolina             US   
0   2020-03-26 00:00:01  45001.0  Abbeville  South Carolina             US   
..                  ...      ...        ...             ...            ...   
779 2020-07-11 00:00:01  55027.0      Dodge       Wisconsin             US   
776 2020-07-12 00:00:01  13091.0      Dodge         Georgia             US   
777 2020-07-12 00:00:01  27039.0      Dodge       Minnesota             US   
778 2020-07-12 00:00:01  31053.0      Dodge        Nebraska             US   
779 2020-07-12 00:00:01  55027.0      Dodge       Wisconsin             US   

             Last_Update        Lat      Long_  Confirmed  Deat